<h1 style='background:#FF008C; font-size:36px; color:white; padding-top:25px;'><center> <b> Tweets Classification with BERT </b> </center> </h1>

![](http://miro.medium.com/max/688/0*B8VDCnh8qBnwUuM4)

> <h3 style='color:#0084FF;'> Importing Required Libraries && tensorflow_text is must for Loading Preprocessor from Tensorflow hub. </h3>

In [ ]:
!pip install tensorflow_text
import tensorflow_text as text

In [ ]:
import re
import string
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

import plotly.express as px

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
raw_data = pd.read_csv('../input/nlp-getting-started/train.csv')

> <h3 style='color:pink; font:16px;'> The Below codes show some data of the CSV Datset </h3>

In [ ]:
raw_data.head().style.set_table_styles(
    [{'selector': 'tr:hover',
      'props': [('background-color', 'green')]}]
).set_properties(**{
    'font-size': '16pt',})

In [ ]:
raw_data.isnull().sum()

**The Below code show some king of basic Preprocessing for Natural Language Preprocessing.**

The Preprocessing steps are as follows:
> 1. Removing Stopwords.
> 2. Removing Punctuations & Symbols.
> 3. Removing Http Code using Beautiful Soup.

In [ ]:
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [ ]:
def sentence_replication(sentence):
    sentence = re.sub(r'http\S+', '', sentence)
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    sentence = sentence.replace("\\", " \\ ")
    sentence = sentence.replace("0", " ")
    sentence = sentence.replace("1", " ")
    sentence = sentence.replace("2", " ")
    sentence = sentence.replace("3", " ")
    sentence = sentence.replace("4", " ")
    sentence = sentence.replace("5", " ")
    sentence = sentence.replace("6", " ")
    sentence = sentence.replace("7", " ")
    sentence = sentence.replace("8", " ")
    sentence = sentence.replace("9", " ")
    soup = BeautifulSoup(sentence)
    sentence = soup.getText()
    return sentence

def PreProcessing(df, training=True):
    
    df['keyword'] = df['keyword'].fillna('')
    df['location'] = df['location'].fillna('')
    
    SENTENCE = []
    LABELS = []
    
    for idx in df.index:
        sentence = raw_data.iloc[idx, 1].lower()
        sentence += ' '
        sentence += raw_data.iloc[idx, 2].lower()
        sentence += ' '
        sentence += raw_data.iloc[idx, 3].lower()
        sentence = sentence_replication(sentence)
        words = sentence.split()
        filtered_sentence = ''
        for word in words:
            word = word.translate(table)
            if word not in stopwords:
                filtered_sentence = filtered_sentence + word + ' '
        SENTENCE.append(filtered_sentence)
        
        if training:
            label = raw_data.iloc[idx, 4]
            LABELS.append(label)
            
    if training:
        return np.array(SENTENCE), np.array(LABELS)
    
    return np.array(SENTENCE)


In [ ]:
X, Y = PreProcessing(raw_data)

![](http://miro.medium.com/max/1200/1*fnfos3Z_xWdK9GdxX_pGxQ.jpeg)

 <h1 style='color:#ff8300; font-size:40px;'> <center> Bidirectional Encoder Representations from Transformers </center> </h1>

<div style='font-size:24px;'>
    
> BERT provides dense vector representations for natural language by using a deep, pre-trained neural network with the Transformer architecture. 

> Along with the Encoder, There is need to load the Bert Preprocessor which take cares of all of the text processing automatically that is being directly feed to the network.    
    
> ***BERT is the Model which is being used by Google in the Google Search Engine.***
    
> It is being trained on millions of wikipedia and other popular websites texts.
    
> Tensorflow provide API to load these Pretrained Models and Layers.
    
</div>

In [ ]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
def build_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
    preprocessed_text = bert_preprocessor(text_input)
    output = bert_encoder(preprocessed_text)
    o = tf.keras.layers.Dropout(0.1, name='Dropout')(output['pooled_output'])
    o = tf.keras.layers.Dense(units=1, activation='sigmoid')(o)
    
    model = tf.keras.Model(inputs=[text_input], outputs=[o])
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

> <h2 style='color:#ff6f00'> Model summary shows that we need to only train the Parameter of our DNN i.e. 769. </h2>

In [ ]:
tweet_model = build_model()
tweet_model.summary()

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2)

In [ ]:
print('Training Examples   : ', x_train.shape)
print('Training Labels     : ', y_train.shape)
print('Validation Examples : ', x_valid.shape)
print('Validation Labels   : ', y_valid.shape)

In [ ]:
hist = tweet_model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=40)

<h1 style='color:#1A00FF;'> <center> Visualization of Model Training Accuracy & Loss </center> </h1>

In [ ]:
hist_df = hist.history
hist_df = pd.DataFrame(hist_df)
fig = px.line(hist_df)
fig.update_layout(template='plotly_dark', width=1200, title='Metrics')
fig.show()

# Test Submission

In [ ]:
test_df = pd.read_csv('../input/nlp-getting-started/test.csv')
id_col = test_df.iloc[:, 0]
x_test = PreProcessing(test_df, training=False)

In [ ]:
pred = tweet_model.predict(x_test)
pred = np.where(pred>=0.5, 1, 0)
pred = pred.reshape(-1)

In [ ]:
df = {'Id':id_col, 'target':pred}
df = pd.DataFrame(df)
df.to_csv('Submission.csv', index=False)

<div style='background:black;'>
    <h1 style='padding:25px 0 0 0; color:#00E9FF'> <center> Thanks for looking ! </center></h1>
    <h1 style='color:#00E9FF;'> <center> Good to hear from You! </center></h1>
</div>